In [1]:
import pandas as pd
import psycopg2 as pg

import database

In [35]:
def agg_factory_SELECT(agg):
    return "{feature}.{agg} AS {feature}_{agg}".format(agg=agg, feature="{feature}")

In [74]:
mean = agg_factory_SELECT('mean')
median = agg_factory_SELECT('median')
aggs = ',\n'.join([mean, std])
print(aggs)

{feature}.mean AS {feature}_mean,
{feature}.std AS {feature}_std


In [47]:
pitches = [f'pitch_{i}' for i in range(12)]
spectrals = "spectral_bandwidth spectral_centroid spectral_flatness spectral_rolloff".split()
tonnetzes = "x_perf5 x_min3 x_maj3 y_perf5 y_min3 y_maj3".split()
features = "tempogram dynamic_tempo rmse zero_crossing_rate".split()
agg_features = features + spectrals + tonnetzes + pitches

In [107]:
def features_aggs_SELECT(features, *aggs):
    aggs = ',\n'.join(agg_factory_SELECT(agg) for agg in aggs)
    return ',\n\n'.join(aggs.format(feature=feature) for feature in features)

In [108]:
def table_others_LEFT_JOIN(table, *others):
    leftjoin = "LEFT JOIN {other} ON {table}.song={other}.song AND " \
               "{table}.segment={other}.segment".format(table=table, other="{other}")  # use () to slit lines?
    
    leftjoins = ",\n".join(leftjoin.format(other=other) for other in others)
    return f'FROM {table}\n{leftjoins}'

In [112]:
# NOTE: treat 'tempo' feature differently since it only has one feature -- 'val'

features_SELECT = features_aggs_SELECT(agg_features, 'mean', 'median', 'mode', 'std')
features_LEFT_JOIN = table_others_LEFT_JOIN("tempo", *agg_features)
features_SELECT_LEFT_JOIN = f"SELECT\n{features_SELECT}\n\n{features_LEFT_JOIN}"

features_SELECT_LEFT_JOIN = f"""
SELECT
tempo.val AS tempo,

{features_SELECT}

{features_LEFT_JOIN};
"""

print(features_SELECT_LEFT_JOIN)


SELECT
tempo.val AS tempo,

tempogram.mean AS tempogram_mean,
tempogram.median AS tempogram_median,
tempogram.mode AS tempogram_mode,
tempogram.std AS tempogram_std,

dynamic_tempo.mean AS dynamic_tempo_mean,
dynamic_tempo.median AS dynamic_tempo_median,
dynamic_tempo.mode AS dynamic_tempo_mode,
dynamic_tempo.std AS dynamic_tempo_std,

rmse.mean AS rmse_mean,
rmse.median AS rmse_median,
rmse.mode AS rmse_mode,
rmse.std AS rmse_std,

zero_crossing_rate.mean AS zero_crossing_rate_mean,
zero_crossing_rate.median AS zero_crossing_rate_median,
zero_crossing_rate.mode AS zero_crossing_rate_mode,
zero_crossing_rate.std AS zero_crossing_rate_std,

spectral_bandwidth.mean AS spectral_bandwidth_mean,
spectral_bandwidth.median AS spectral_bandwidth_median,
spectral_bandwidth.mode AS spectral_bandwidth_mode,
spectral_bandwidth.std AS spectral_bandwidth_std,

spectral_centroid.mean AS spectral_centroid_mean,
spectral_centroid.median AS spectral_centroid_median,
spectral_centroid.mode AS spectral_c

In [3]:
connection = database.connect_db()

In [61]:
# use atom ctr-D to easily copy-paste new fields
query = """
SELECT
    pitch_0.mean AS p0_mean,
    pitch_0.mode AS p0_mode,
    pitch_0.median AS p0_median,
    pitch_0.amax AS p0_max,
    pitch_0.amin AS p0_min,
    pitch_0.std AS p0_std,
    
    pitch_1.mean AS p1_mean,
    
    pitch_2.mean AS p2_mean,
    
    pitch_3.mean AS p3_mean,
    
    pitch_4.mean AS p4_mean,
    
    pitch_5.mean AS p5_mean,
    
    pitch_6.mean AS p6_mean,
    
    pitch_7.mean AS p7_mean,
    
    pitch_8.mean AS p8_mean,
    
    pitch_9.mean AS p9_mean,
    
    pitch_10.mean AS p10_mean,
    
    pitch_11.mean AS p11_mean
    
    
FROM pitch_0
LEFT JOIN pitch_1  ON pitch_0.song = pitch_1.song  AND pitch_0.segment = pitch_1.segment
LEFT JOIN pitch_2  ON pitch_0.song = pitch_2.song  AND pitch_0.segment = pitch_2.segment
LEFT JOIN pitch_3  ON pitch_0.song = pitch_3.song  AND pitch_0.segment = pitch_3.segment
LEFT JOIN pitch_4  ON pitch_0.song = pitch_4.song  AND pitch_0.segment = pitch_4.segment
LEFT JOIN pitch_5  ON pitch_0.song = pitch_5.song  AND pitch_0.segment = pitch_5.segment
LEFT JOIN pitch_6  ON pitch_0.song = pitch_6.song  AND pitch_0.segment = pitch_6.segment
LEFT JOIN pitch_7  ON pitch_0.song = pitch_7.song  AND pitch_0.segment = pitch_7.segment
LEFT JOIN pitch_8  ON pitch_0.song = pitch_8.song  AND pitch_0.segment = pitch_8.segment
LEFT JOIN pitch_9  ON pitch_0.song = pitch_9.song  AND pitch_0.segment = pitch_9.segment
LEFT JOIN pitch_10 ON pitch_0.song = pitch_10.song AND pitch_0.segment = pitch_10.segment
LEFT JOIN pitch_11 ON pitch_0.song = pitch_11.song AND pitch_0.segment = pitch_11.segment;
"""

In [ ]:
result = cursor.execute(query)

In [10]:
df = pd.read_sql(query, connection)

In [11]:
df.head()

,p0_mean,p1_mean,p2_mean,p3_mean,p4_mean,p5_mean,p6_mean,p7_mean,p8_mean,p9_mean,p10_mean,p11_mean
0,0.478847,0.396772,0.125975,0.122057,0.182186,0.473038,0.529897,0.167450,0.168881,0.158108,0.323886,0.237460
1,0.535089,0.460853,0.153411,0.151650,0.216026,0.497114,0.575560,0.228770,0.219865,0.219131,0.368779,0.272720
2,0.532976,0.458424,0.156275,0.176226,0.216139,0.513915,0.563739,0.233496,0.231977,0.233574,0.395313,0.293611
3,0.560920,0.466850,0.166367,0.164566,0.213470,0.491722,0.558483,0.228801,0.232974,0.227487,0.386434,0.293144
4,0.553595,0.455063,0.161908,0.164667,0.226997,0.517006,0.575496,0.243437,0.243798,0.274195,0.440270,0.306196


In [12]:
connection.close()

In [13]:
del df